<a href="https://colab.research.google.com/github/FERNANDOBUCHELLY/ejemplo/blob/main/Parcial1_SyS_2024_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Parcial 1: Señales y Sistemas 2024-II

 ## Profesor: Andrés Marino Álvarez Meza, Ph.D.


## Departamento de Ingeniería Eléctrica, Electrónica, y Computación
## Universidad Nacional de Colombia - sede Manizales

# Instrucciones

-- Para recibir el crédito total, sus respuestas deben estar justificadas de manera clara, detallada y concreta, mostrando los procedimientos y razonamientos paso a paso.

-- Está permitido el uso de herramientas de inteligencia artificial (IA). Si las utiliza, por favor declare explícitamente cómo fueron empleadas en la resolución de cada pregunta. Incluya los prompts (consultas) y las iteraciones realizadas con las IA durante el desarrollo del parcial.

-- La entrega del parcial debe realizarse antes de las 23:59 del 5 de diciembre de 2024 al correo electrónico amalvarezme@unal.edu.co mediante un enlace de GitHub.

-- Los códigos deben estar debidamente comentados en las celdas correspondientes y explicados en celdas de texto (markdown). Los códigos que no incluyan comentarios ni discusiones no serán considerados en la evaluación final.

# Pregunta 1 (valor 2.5 puntos)

Cuál es la señal obtenida en tiempo discreto al utilizar un conversor análogo digital de 5 bits con frecuencia de muestreo de $5kHz$, entrada análoga de -3.3 a 3.3 [v], aplicado a la señal continua $x(t) = 0.3 \cos(1000\pi t-\pi/4) +
0.6 \sin(2000\pi t) + 0.1 \cos(11000\pi t-\pi)$?. Realizar la simulación del proceso de digitalización incluyendo al menos 3 ciclos de la señal $x(t)$.

En caso de que la digitalización no sea apropiada, diseñe e implemente un conversor adecuado para la señal estudiada. El convesor debe permitir configurar la cantidad de bits, rango de la entrada análoga y la frecuencia de muestreo, indicándole al usuario si dicha frecuencia es apropiada o no, y graficar la señal continua, discreta y digital.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def x(t):
  return 0.3*np.cos(1000*np.pi*t-np.pi/4) + 0.6*np.sin(2000*np.pi*t) + 0.1*np.cos(11000*np.pi*t-np.pi)

fo = 5500
fs = (2)*fo
Ts = 1/fs
To = 1/500

# Define tv and y before using them
tv = np.arange(0, 3*To, Ts)  # Vector de tiempo
y = x(tv)

# Definir la función my_ceropen (cero y pendiente)
def my_ceropen(x, ymin=0, ymax=200):
  xmax = max(x)
  xmin = min(x)
  m = (ymax - ymin) / (xmax - xmin)
  c = ymin - m * xmin
  yv = m * x + c
  return yv

  #cero y pendiente
ycs = my_ceropen(y, ymin=-3.3,ymax=3.3)

#número de bits y vector de cuantización
nbits = 5
rmin = -3.3
rmax = 3.3
ve = np.linspace(rmin,rmax,2**nbits) # (rmax-rmin)/(2^#bits - 1) -> si se quiere incluir rmin y rmax
from scipy.spatial.distance import cdist
# Función de cuantización
def my_cuantizador(yn, vq):
  dn = cdist(yn.reshape(-1, 1), vq.reshape(-1, 1))
  ind = np.argmin(dn)
  return vq[ind]

# Cuantizar la señal
yq = np.zeros_like(ycs)
for i in range(len(ycs)):
  yq[i] = my_cuantizador(ycs[i], ve)

# Graficar
plt.plot(tv, ycs, c='black', label='ycs') # Señal original
plt.stem(tv, ycs, label='ycs_dis') # Señal discretizada
plt.plot(tv, yq, c='y', marker='o', label='ydig', linewidth=2)  # Señal cuantificada
# Mostrar niveles de cuantificación
for v in ve:
  plt.plot(tv,v*np.ones(len(tv)))
plt.legend()
plt.grid()
plt.xlabel('t')
plt.ylabel('Amplitud')
plt.title('Señal Original, Discretizada y Cuantificada')
plt.show()



# Pregunta 2 (valor 2.5 puntos)

Se dispone de un sistema modelado como una "caja negra" (ver celdas de código). Su tarea es analizar y comprobar mediante simulaciones si el sistema cumple con las propiedades de linealidad e invariancia en el tiempo. En caso de que el sistema sea lineal e invariante con el tiempo, determine su respuesta al impulso y utilice esta respuesta para calcular la salida del sistema ante la siguiente señal:

$x[n] = \sin[100 \pi n ] + \sin[600 \pi n]$

In [ ]:
# cargar sistema
FILEID = "1J9rhh0wWHZSBd8XmWGt1ZpCsMDuoUFmm"
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id='$FILEID -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id="$FILEID -O P1_model.zip && rm -rf /tmp/cookies.txt
!unzip -o P1_model.zip
!dir

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import joblib
from P1_model import system_
from scipy.signal import firwin, freqz, lfilter, filtfilt

#sistema pregunta 2
my_system = system_.My_System()
my_system.create_()
fs = my_system.fs #frecuencia de muestreo
t = np.arange(-0.01, 0.02, 1/fs)  # Tiempo
signal_u = np.heaviside(t,1) # función heaviside
y_u = my_system.predict(signal_u)


# Visualización de las señales
fig, axs = plt.subplots(2,1)
axs[0].stem(t, signal_u, label='Señal de entrada')
axs[0].set_xlabel('Tiempo (s)')
axs[0].set_ylabel('Amplitud')
axs[0].legend()
axs[0].grid()
axs[1].stem(t,y_u, label='Señal salida')
axs[1].set_xlabel('Tiempo (s)')
axs[1].set_ylabel('Amplitud')
axs[1].legend()
axs[1].grid()
plt.tight_layout()
plt.show()

In [ ]:
y_ax = my_system.predict(signal_u) + my_system.predict(5 * signal_u)  # Se suman señales para comprobar superposición
signal_k = np.heaviside(t - 0.01, 1)  # Se traslada la entrada para ver si traslada la salida
y_k = my_system.predict(signal_k)

# Verificar si el sistema es lineal
y_ax_pred = my_system.predict(signal_u + 5 * signal_u)  # Salida esperada para la suma de señales
lineal = np.allclose(y_ax, y_ax_pred)  # Verificar si y_ax coincide con y_ax_pred

# Graficar los resultados
fig, axs = plt.subplots(2, 1)
axs[0].stem(t, y_ax, label='Propiedad de superposición')
axs[0].set_xlabel('Tiempo (s)')
axs[0].set_ylabel('Amplitud')
axs[0].legend()
axs[0].grid()
axs[1].stem(t, y_k, label='Propiedad de traslación')
axs[1].set_xlabel('Tiempo (s)')
axs[1].set_ylabel('Amplitud')
axs[1].legend()
axs[1].grid()
plt.tight_layout()
plt.show()

# Imprimir resultado de linealidad
print(f"¿Es lineal (cumple superposición)? {'Sí' if lineal else 'No'}")


In [ ]:
# Verificar invariancia en el tiempo
signal_k = np.heaviside(t - 0.01, 1)  # Señal Heaviside desplazada en el tiempo
y_k = my_system.predict(signal_k)  # Salida del sistema para la señal desplazada
y_expected = np.roll(my_system.predict(signal_u), int(0.01 * fs))  # Salida esperada desplazada

# Comparar salida real vs salida esperada para determinar invariancia
invariante = np.allclose(y_k, y_expected)  # True si coinciden

# Graficar los resultados
fig, axs = plt.subplots(2, 1)
axs[0].stem(t, my_system.predict(signal_u), label='Salida original (no desplazada)')
axs[0].set_xlabel('Tiempo (s)')
axs[0].set_ylabel('Amplitud')
axs[0].legend()
axs[0].grid()
axs[1].stem(t, y_k, label='Salida desplazada (real)')
axs[1].plot(t, y_expected, '--', label='Salida desplazada (esperada)', color='r')  # Línea discontinua para comparación
axs[1].set_xlabel('Tiempo (s)')
axs[1].set_ylabel('Amplitud')
axs[1].legend()
axs[1].grid()
plt.tight_layout()
plt.show()

# Imprimir resultado de invariancia
print(f"¿Es invariante en el tiempo? {'Sí' if invariante else 'No'}")


In [ ]:
# Calcular la respuesta al impulso
delta = np.zeros_like(t)  # Crear una señal delta (impulso unitario)
idx_zero = np.argmin(np.abs(t))  # Encontrar el índice más cercano a t=0
delta[idx_zero] = 1  # Colocar el impulso en el tiempo cero
h = my_system.predict(delta)  # Obtener la respuesta al impulso del sistema

# Graficar la respuesta al impulso
plt.figure(figsize=(8, 5))
plt.stem(t, h, label='Respuesta al impulso h[n]')
plt.xlabel('Tiempo (s)')
plt.ylabel('Amplitud')
plt.title('Respuesta al Impulso del Sistema')
plt.legend()
plt.grid()
plt.tight_layout()
plt.show()


In [ ]:
# Definir la señal de entrada x[n]
n = np.arange(len(t))  # Define n como una matriz de indice en el tiempo
x_n = np.sin(100 * np.pi * n / fs) + np.sin(600 * np.pi * n / fs)  # La señal x[n] = sin(100πn) + sin(600πn)

# Realizar la convolución
y_convolucion = np.convolve(x_n, h, mode='same')  # 'same' para obtener la salida con el mismo tamaño que x[n]

# Graficar los resultados
plt.figure(figsize=(8, 5))
plt.plot(t, y_convolucion, label='Salida de la convolución')
plt.xlabel('Tiempo (s)')
plt.ylabel('Amplitud')
plt.title('Salida del Sistema utilizando Convolución')
plt.legend()
plt.grid()
plt.tight_layout()
plt.show()
